# 20_GeoTIFF Alignment
This notebook ensures all raw GeoTIFF layers are spatially aligned to a consistent grid before feature extraction and modeling. Geophysical datasets (e.g., gravity, magnetics, radiometrics) often come in different:
- CRS (Coordinate Reference Systems)
- Spatial extent and resolution
- Grid alignment

To ensure reliable analysis and feature stacking, all input rasters must be reprojected to the same CRS, resolution, bounds, and pixel layout.

## What This Notebook Does
- Sets a common target grid:
    - CRS: EPSG:3577 (Australian Albers, meters)
    - Resolution: 250m
    - Extent: Covers all of mainland Australia

Aligns all .tif files in the /data/raw/ directory to this grid using rasterio.reproject.

Outputs aligned rasters to /data/aligned/, ready for feature engineering and model input.

Includes optional side-by-side visualization (original vs. aligned) for quality check.

## Why need alignment
Spatial misalignment between layers can lead to incorrect feature values at sampling points and inaccurate model predictions. Alignment is a critical preprocessing step to guarantee consistency across datasets.

In [ ]:
import os
import math
import rasterio
from rasterio.warp import reproject, Resampling, calculate_default_transform
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Transformer
from tqdm import tqdm

# === 1. Configuration ===
RAW_DIR = "../../data/raw"
ALIGNED_DIR = "../../data/aligned"
os.makedirs(ALIGNED_DIR, exist_ok=True)

TARGET_CRS = "EPSG:3577"  # GDA94 / Australian Albers
RESOLUTION = 250  # meters
NODATA_VALUE = -9999

# Geographic bounds (EPSG:4326)
min_lon, min_lat = 113.75, -43.3
max_lon, max_lat = 153.55, -12.4

# === 2. Transform bounds to target CRS ===
transformer = Transformer.from_crs("EPSG:4326", TARGET_CRS, always_xy=True)
xmin, ymin = transformer.transform(min_lon, min_lat)
xmax, ymax = transformer.transform(max_lon, max_lat)

# === 3. Alignment function ===
def align_with_rasterio(input_path, output_path):
    try:
        with rasterio.open(input_path) as src:
            
            transform, width, height = calculate_default_transform(
                src.crs, TARGET_CRS, src.width, src.height, *src.bounds,
                resolution=RESOLUTION
            )
            
            
            kwargs = src.meta.copy()
            kwargs.update({
                'crs': TARGET_CRS,
                'transform': transform,
                'width': width,
                'height': height,
                'driver': 'GTiff',
                'compress': 'lzw',
                'nodata': NODATA_VALUE
            })
            
            with rasterio.open(output_path, 'w', **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=TARGET_CRS,
                        resampling=Resampling.bilinear
                    )
                
        print(f"Aligned: {os.path.basename(input_path)}")
        return True
    except Exception as e:
        print(f"Error aligning {input_path}: {e}")
        return False

# === 4. Batch processing ===
input_files = [f for f in os.listdir(RAW_DIR) if f.lower().endswith(".tif")]

for fname in tqdm(input_files, desc="Aligning rasters"):
    input_path = os.path.join(RAW_DIR, fname)
    name, ext = os.path.splitext(fname)
    output_path = os.path.join(ALIGNED_DIR, f"{name}_aligned{ext}")
    align_with_rasterio(input_path, output_path)

Aligning rasters:   7%|▋         | 1/14 [00:40<08:51, 40.87s/it]

Aligned: gravity_cscba.tif


Aligning rasters:  14%|█▍        | 2/14 [01:20<08:01, 40.09s/it]

Aligned: gravity_cscba_1vd.tif


Aligning rasters:  21%|██▏       | 3/14 [01:42<05:52, 32.01s/it]

...

Aligned: radio_u_k_ratio.tif


Aligning rasters:  93%|█████████▎| 13/14 [15:23<01:13, 73.40s/it]

Aligned: radio_u_ppm.tif


Aligning rasters: 100%|██████████| 14/14 [16:34<00:00, 71.07s/it]

Aligned: radio_u_th_ratio.tif
